In [ ]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var 
import skimage.io
from pathlib import Path
import numpy
from PIL import Image

In [ ]:
#### part I: Loading your data
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True)


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw




Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [ ]:
### Part I : Loading Your Data 

T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

In [ ]:
#### Part II : Writing the Network
class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.cnn1 = nn.Conv2d(1,3,3)
    self.cnn2 = nn.Conv2d(3,2,5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x

In [ ]:
#### Part III : Writing the main Training loop

mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr = learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100.
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.cuda())
    x = model(images)
    value,pred = torch.max(x,1)
    pred = pred.data.cpu()
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100./total

for e in range(n_epoch):
  for i,(images,labels) in enumerate(train_dl):
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

Epoch : 1 Batch : 10 Loss : 2.04787015914917 Accuracy : 51.209999084472656 %
Epoch : 1 Batch : 20 Loss : 0.7888721227645874 Accuracy : 77.61000061035156 %
Epoch : 1 Batch : 30 Loss : 0.30716848373413086 Accuracy : 86.19999694824219 %
Epoch : 1 Batch : 40 Loss : 0.21726086735725403 Accuracy : 87.79000091552734 %
Epoch : 1 Batch : 50 Loss : 0.17807471752166748 Accuracy : 89.12999725341797 %
Epoch : 1 Batch : 60 Loss : 0.20550251007080078 Accuracy : 90.51000213623047 %
Epoch : 1 Batch : 70 Loss : 0.26585277915000916 Accuracy : 92.62999725341797 %
Epoch : 1 Batch : 80 Loss : 0.21038389205932617 Accuracy : 92.04000091552734 %
Epoch : 1 Batch : 90 Loss : 0.21131452918052673 Accuracy : 91.30999755859375 %
Epoch : 1 Batch : 100 Loss : 0.1501646190881729 Accuracy : 92.69000244140625 %
Epoch : 1 Batch : 110 Loss : 0.46894946694374084 Accuracy : 92.48999786376953 %
Epoch : 1 Batch : 120 Loss : 0.1493590772151947 Accuracy : 93.62000274658203 %
Epoch : 1 Batch : 130 Loss : 0.2843298316001892 Accura

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
no_of_digits = 81
sudoku_grid = numpy.zeros((9,9))
for i in range(no_of_digits):
  ip_image = Image.open(str(i)+'.jpg')
  if numpy.mean(ip_image) != 0:
    image_2_npArray = numpy.asarray(ip_image)
    image_2_npArray_2_tensor = torchvision.transforms.ToTensor()(image_2_npArray)
    image_2_npArray_2_tensor = image_2_npArray_2_tensor.unsqueeze(0).to(device)
    pred = mycnn(image_2_npArray_2_tensor).cpu().detach().numpy()
    print(f'i: {i}')
    print(f'vector: {pred}')
    print(f'prediction: {numpy.argmax(pred)}')
    row_no = int(i/9)
    col_no = i-row_no*9
    sudoku_grid[row_no, col_no] = numpy.argmax(pred)
  else:
    pass

i: 3
vector: [[-13.581383  -12.485875   -2.3304002 -14.196305   -8.227797   -2.3409367
   -2.1966665 -13.124601   -6.4497333 -13.324231 ]]
prediction: 6
i: 5
vector: [[-9.022875  -5.942332   0.9879993 -9.323812  -5.205147  -7.338716
  -1.247893  -7.4808497 -1.4751291 -5.3941507]]
prediction: 2
i: 6
vector: [[ -9.56928    -9.536061   -2.6761205  -3.5256171 -11.213648   -8.631451
  -24.889473   16.498344   -2.4498162   1.7516643]]
prediction: 7
i: 9
vector: [[ -6.917219  -11.515067    3.023043   -1.377013  -24.84398   -10.0675535
  -25.401709    8.78768     2.5277267  -1.9914082]]
prediction: 7
i: 11
vector: [[-11.160907   -22.602156   -10.544547   -12.346034   -10.868121
    0.30970293  -3.0285618  -12.683837     2.2496238  -10.012836  ]]
prediction: 8
i: 17
vector: [[-10.714184  -22.868628   -7.1169753  -0.4063931  -9.849643   -8.0297165
  -12.40434   -13.105727   -2.4174056  -9.127984 ]]
prediction: 3
i: 23
vector: [[-17.09589   -28.66431   -21.46915    -1.3329237 -14.870318    7.2311

In [ ]:
print(sudoku_grid)

[[0. 0. 0. 6. 0. 2. 7. 0. 0.]
 [7. 0. 8. 0. 0. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0. 5. 0. 8. 0.]
 [0. 7. 0. 0. 7. 0. 0. 9. 2.]
 [8. 0. 0. 0. 0. 0. 0. 0. 5.]
 [6. 8. 0. 0. 0. 0. 0. 2. 0.]
 [0. 5. 0. 2. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 2. 9. 8.]
 [0. 0. 2. 8. 0. 0. 0. 0. 0.]]
